In [3]:
import numpy as np
import json
import os
import argparse
import pandas as pd
import logging
import telethon

# Choosing the seed channels

You need paid account to access the data in table form, but I'm not spending even a cent on this russian service:)
I copypasted the tables from webpage directly to [google spreadsheet](https://docs.google.com/spreadsheets/d/1J6oCBaG2JE5ChDI5E018pRGJ9c9cnOZCOtW0iOeMwHw/edit#gid=0) and then extract links from cells using [custom App Script](https://docs.google.com/spreadsheets/d/1J6oCBaG2JE5ChDI5E018pRGJ9c9cnOZCOtW0iOeMwHw/edit#gid=0)

I've selected top100 (by citation index) channels from News and Media section, top100 from Blogs and top100 from Politics. I chose citation index (which is based on forwards and mentions) because it reflects potential information diffusion through the whole network (and we'll rely on the measure of influence further on).

In [6]:
df_links_raw = pd.read_csv("/Users/katerynaburovova/PycharmProjects/dehumanization/data/links_raw.csv")

In [7]:
df_links_raw = df_links_raw[df_links_raw['Unnamed: 2']!='drop'].reset_index(drop=True)

We have 10 values that were not linked through the share link - those are private and can be accessed after joining and parsing all joined chats. I expect that they will appear in our referenced chat - we drop it for now.

In [8]:
df_links_raw = df_links_raw[df_links_raw['name_handle']!='#VALUE!']

In [9]:
df_links_raw = (df_links_raw.drop(columns=['Unnamed: 2', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 5']))

We don't have duplicates:

In [10]:
df_links_raw['name_handle'].value_counts()[:5]

rian_ru         1
boris_rozhin    1
RKadyrov_95     1
SolovievLive    1
russica2        1
Name: name_handle, dtype: int64

In [16]:
list_of_names = df_links_raw['name_handle'].to_list()

## Now we create json which we'll feed to the parsing sript.

In [17]:
dict = {"names": list_of_names}


In [18]:
json_object = json.dumps(dict, indent=4)

In [19]:
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels_list.json", "w") as outfile:
    outfile.write(json_object)

In [20]:
# bs check
with open('/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels_list.json', 'r') as openfile:
     data = json.load(openfile)
data['names'][0]

'rian_ru'

In [24]:
# import os
# import sys
# os.chdir(sys.path[0])
#
# if f"{'tmp'}.session" in os.listdir():
#     os.remove(f"{'tmp'}.session")

In [1]:
dict2 = {"names": ["readovkanews"]}

In [4]:
json_object = json.dumps(dict2, indent=4)


In [5]:
with open("/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels_list2.json", "w") as outfile:
    outfile.write(json_object)

In [26]:
import numpy as np
import json
import os
import argparse
import pandas as pd
import logging
import telethon
import glob

In [21]:
with open('/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels_list.json', 'r') as openfile:
     data = json.load(openfile)
data['names'][0]

'rian_ru'

In [24]:
len(data['names'])

300

In [27]:
loaded_channels_path = "/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels"
channel_data_files = glob.glob(f"{loaded_channels_path}/*.csv")

In [28]:
channel_data_files

['/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels/wargonzo.csv',
 '/Users/katerynaburovova/PycharmProjects/dehumanization/data/channels/readovkanews.csv']

In [30]:
already_loaded_list = []
for channel in channel_data_files:
    local_df = pd.read_csv(channel)
    already_loaded_list.append(os.path.basename(channel).split(".")[0])

In [31]:
already_loaded_list

['wargonzo', 'readovkanews']

In [32]:
current_list = set(data['names'])-set(already_loaded_list)

In [34]:
len(current_list)

298

In [37]:
current_list = list(current_list)

In [38]:
dict2 = {"names": current_list}
json_object = json.dumps(dict2, indent=4)